# KG 성능 비교 실험 (ROUGE 기반 평가)


In [1]:
import pandas as pd
from rouge_score import rouge_scorer

# 데이터 불러오기
kg = pd.read_csv("kg_triples_test.csv")
test_df = pd.read_csv("test.csv")

# 트리플 결합 텍스트 만들기
kg["triple_text"] = kg["subject"] + " " + kg["predicate"] + " " + kg["object"]


In [2]:
# 간단한 유사도 기반 KG 검색 함수
def find_relevant_kg(text, kg_texts, top_k=3):
    matches = []
    for t in kg_texts:
        score = sum([1 for w in t.split() if w in text])
        matches.append((t, score))
    matches.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in matches[:top_k]]


In [3]:
# 프롬프트 구성 함수
def build_prompt(text, kg_hits):
    context = "\n".join(kg_hits)
    return f"다음은 참고 지식입니다:\n{context}\n\n사용자 입력: {text}\n답변:"


In [4]:
# ROUGE 평가 함수
def evaluate_with_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred)["rougeL"].fmeasure for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)


In [5]:
from random import randint

# 샘플 10개 추출
sample = test_df.sample(10, random_state=42)

# 1. KG 없이 (baseline)
preds_no_kg = ["답변: " + text[:20] for text in sample["input"]]

# 2. KG 사용
preds_with_kg = []
for text in sample["input"]:
    hits = find_relevant_kg(text, kg["triple_text"])
    prompt = build_prompt(text, hits)
    preds_with_kg.append("답변: " + text[:20] + " (지식반영)")

# 평가
rouge_no_kg = evaluate_with_rouge(preds_no_kg, sample["output"])
rouge_with_kg = evaluate_with_rouge(preds_with_kg, sample["output"])

print(f"KG 미사용 시 ROUGE-L: {rouge_no_kg:.4f}")
print(f"KG 사용 시 ROUGE-L: {rouge_with_kg:.4f}")


KeyError: 'input'

In [24]:
# ✅ 1. 필요한 라이브러리 로딩
import pandas as pd
import subprocess

# ✅ 2. Knowledge Graph 로딩 및 텍스트 구성
kg = pd.read_csv("kg_triples_test.csv")
kg["triple_text"] = kg["subject"] + " " + kg["predicate"] + " " + kg["object"]

# ✅ 3. 테스트 질문 로딩
test_df = pd.read_csv("test.csv")

# ✅ 4. KG 검색 함수
def find_relevant_kg(user_input, kg_texts, topk=3):
    return [t for t in kg_texts if any(word in user_input for word in t.split())][:topk]

# ✅ 5. 프롬프트 생성 함수
def build_prompt(user_input, kg_hits=None):
    prompt = f"질문: {user_input}\n"
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += "답변:"
    return prompt

# ✅ 6. 로컬 Ollama 모델 호출 함수 (예: llama3)
def run_ollama(prompt, model="llama3"):
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        timeout=30
    )
    return result.stdout.decode("utf-8").strip()

# ✅ 7. 질문 입력 → KG 사용 여부에 따른 비교
sample = test_df.sample(1, random_state=42)
user_input = sample["sentence1"].values[0]
true_answer = sample["sentence2"].values[0]

kg_hits = find_relevant_kg(user_input, kg["triple_text"])
prompt_kg = build_prompt(user_input, kg_hits)
prompt_no_kg = build_prompt(user_input)

print("📌 질문:", user_input)
print("\n[프롬프트 - KG 미사용]\n", prompt_no_kg)
print("\n[프롬프트 - KG 사용]\n", prompt_kg)

# ✅ 8. Ollama 실행 및 출력 결과
print("\n🤖 [KG 미사용 답변]\n")
print(run_ollama(prompt_no_kg))

print("\n🤖 [KG 사용 답변]\n")
print(run_ollama(prompt_kg))

📌 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.

[프롬프트 - KG 미사용]
 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.
답변:

[프롬프트 - KG 사용]
 질문: 라이스 페이퍼의 두께나 구성이 일반적이지 않아 조금 놀랐습니다. 아마, 식사량을 충분히 하는 것이 목표였을건데, 이로 인해 월남쌈의 질감이나 맛을 느끼는 부분에서 조금 힘들었습니다. 개인적으로는 훠궈 육수는 조금 아쉬움이 남았지만, 다른 메뉴들 중에서는 고기가 빼어난 맛을 보였습니다.
배경지식:
- 사용자 질문한다 다른 사람의 의도
답변:

🤖 [KG 미사용 답변]



FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd

# CPU로 강제 설정
device = torch.device("cpu")
torch.set_default_tensor_type(torch.FloatTensor)

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForMaskedLM.from_pretrained('monologg/kobert')
model.to(device)
model.eval()

# KG 데이터 불러오기
kg_df = pd.read_csv("kg_triples_test.csv")
if 'triple_text' not in kg_df.columns:
    kg_df['triple_text'] = kg_df['subject'] + " " + kg_df['predicate'] + " " + kg_df['object']

# 배경지식 찾기
def find_relevant_kg(question, kg_df):
    hits = []
    for triple in kg_df['triple_text']:
        if any(word in triple for word in question.split()):
            hits.append(triple)
    return hits[:3]

# 프롬프트 생성
def build_prompt(question, kg_hits=None):
    prompt = ""
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += f"질문: {question}"
    return prompt

# 답변 생성
def generate_answer(prompt):
    if "[MASK]" not in prompt:
        return "질문에 [MASK]가 포함되어야 합니다."
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = outputs.logits[0, mask_token_index, :]
    top_token = torch.argmax(mask_token_logits, dim=1)
    predicted_token = tokenizer.decode(top_token)
    
    return prompt.replace("[MASK]", predicted_token)

# 사용자 입력
user_question = input("질문을 입력하세요 (예: 나는 [MASK] 기분이다): ")

# KG 미사용
prompt_no_kg = build_prompt(user_question)
answer_no_kg = generate_answer(prompt_no_kg)

# KG 사용
kg_hits = find_relevant_kg(user_question, kg_df)
prompt_with_kg = build_prompt(user_question, kg_hits)
answer_with_kg = generate_answer(prompt_with_kg)

print("\n KG 미사용:")
print("프롬프트:", prompt_no_kg)
print("답변:", answer_no_kg)

print("\n KG 사용:")
print("프롬프트:", prompt_with_kg)
print("답변:", answer_with_kg)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📌 KG 미사용:
프롬프트: 질문: 질문: 나는 [MASK] 기분이다
답변: 질문: 질문: 나는 꽉 기분이다

📌 KG 사용:
프롬프트: 질문: 질문: 나는 [MASK] 기분이다
답변: 질문: 질문: 나는 꽉 기분이다


In [ ]:
import subprocess
import pandas as pd

# Ollama 실행 경로 (직접 지정)
OLLAMA_PATH = r"C:\Users\lsm40\AppData\Local\Programs\Ollama\ollama.exe"

# 사용자 질문 입력
user_input = input("질문을 입력하세요: ")

# KG 파일 불러오기
kg_df = pd.read_csv("kg_triples_test.csv")  # 경로는 필요에 따라 수정

# KG에서 관련 정보 추출 (간단한 키워드 기반 검색)
def find_relevant_kg(question, kg_df):
    hits = []
    for idx, row in kg_df.iterrows():
        triple = f"{row['subject']}, {row['predicate']}, {row['object']}"
        if any(word in triple for word in question.split()):
            hits.append(triple)
    return hits[:3]  # 최대 3개까지만 사용

# 프롬프트 생성
def build_prompt(user_input, kg_hits=None):
    prompt = ""
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += f"질문: {user_input}\n답변:"
    return prompt

#  Llama3로 답변 생성
def run_ollama(prompt, model="llama3"):
    try:
        result = subprocess.run(
            [OLLAMA_PATH, "run", model],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=60
        )
        return result.stdout.decode("utf-8").strip()
    except FileNotFoundError:
        return " Ollama 실행 파일을 찾을 수 없습니다. 경로를 다시 확인하세요."
    except subprocess.TimeoutExpired:
        return " 실행 시간이 초과되었습니다."
    except Exception as e:
        return f" 오류 발생: {e}"

#  KG 검색 결과 가져오기
kg_hits = find_relevant_kg(user_input, kg_df)

#  프롬프트 생성
prompt_kg = build_prompt(user_input, kg_hits)
prompt_no_kg = build_prompt(user_input)

#  출력
print("\n [KG 미사용 프롬프트]\n", prompt_no_kg)
print("\n [KG 미사용 답변]\n", run_ollama(prompt_no_kg))

print("\n [KG 사용 프롬프트]\n", prompt_kg)
print("\n [KG 사용 답변]\n", run_ollama(prompt_kg))



📌 [KG 미사용 프롬프트]
 질문: 피곤해
답변:

🤖 [KG 미사용 답변]
 ⏱️ 실행 시간이 초과되었습니다.

📌 [KG 사용 프롬프트]
 질문: 피곤해
답변:

🤖 [KG 사용 답변]
 ⏱️ 실행 시간이 초과되었습니다.


In [ ]:
import subprocess
import pandas as pd
import time

#  Ollama 실행 파일 경로 (직접 확인된 경로로 수정 필요)
OLLAMA_PATH = r"C:\Users\lsm40\AppData\Local\Programs\Ollama\ollama.exe"

#  사용자 질문 실시간 입력
user_input = input("질문을 입력하세요: ").strip()

#  KG 그래프 CSV 로딩 (컬럼명 자동 감지)
kg_path = "./kg_triples_test.csv"  # 예: 'triple_text' 열이 있는 파일
kg_df = pd.read_csv(kg_path)
col = kg_df.columns[0]  # 첫 번째 열 자동 인식

#  KG에서 관련된 배경지식 추출 함수
def find_relevant_kg(question, df, column):
    hits = []
    for triple in df[column]:
        if any(word in triple for word in question.split()):
            hits.append(triple)
    return hits[:3]

#  프롬프트 생성 함수
def build_prompt(question, kg_hits=None):
    prompt = ""
    if kg_hits:
        prompt += "배경지식:\n"
        for hit in kg_hits:
            prompt += f"- {hit}\n"
    prompt += f"질문: {question}\n답변:"
    return prompt

#  Llama3 실행 함수
def run_ollama(prompt, timeout=60):
    try:
        result = subprocess.run(
            [OLLAMA_PATH, "run", "llama3"],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=timeout
        )
        if result.stderr:
            print(" stderr:", result.stderr.decode("utf-8").strip())
        return result.stdout.decode("utf-8").strip()
    except subprocess.TimeoutExpired:
        return " 실행 시간이 초과되었습니다."
    except FileNotFoundError:
        return " Ollama 실행 파일을 찾을 수 없습니다. 경로를 확인하세요."

#  KG 기반 프롬프트 구성
kg_hits = find_relevant_kg(user_input, kg_df, col)
prompt_no_kg = build_prompt(user_input)
prompt_kg = build_prompt(user_input, kg_hits)

#  실행 및 결과 출력
print("\n [KG 미사용 프롬프트]\n", prompt_no_kg)
print("\n [KG 미사용 답변]\n", run_ollama(prompt_no_kg))

print("\n [KG 사용 프롬프트]\n", prompt_kg)
print("\n [KG 사용 답변]\n", run_ollama(prompt_kg))



📌 [KG 미사용 프롬프트]
 질문: 나 피곤해
답변:

🤖 [KG 미사용 답변]
 ⏱️ 실행 시간이 초과되었습니다.

📌 [KG 사용 프롬프트]
 질문: 나 피곤해
답변:

🤖 [KG 사용 답변]
 ⏱️ 실행 시간이 초과되었습니다.


In [40]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import json

# ✅ KoBERT 로드
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)
device = torch.device("cpu")

# ✅ 데이터셋 불러오기
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

data = load_jsonl("poetry.jsonl")
df = pd.DataFrame(data)

# ✅ 라벨 인코딩: "human" → 0, "ai" → 1
label_map = {"human": 0, "ai": 1}
df['label'] = df['label'].map(label_map)

# ✅ NaN 제거 (매핑되지 않은 라벨 있는 행 제거)
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# ✅ KG 불러오기
kg_df = pd.read_csv("kg_triples_test.csv")
kg_texts = (
    kg_df
    .apply(lambda row: f"{row['subject']} {row['predicate']} {row['object']}", axis=1)
    .tolist()
)

# ✅ KG 관련 배경지식 찾기 (간단 키워드 매칭)
def find_relevant_kg(text, kg_texts, topk=3):
    hits = [kg for kg in kg_texts if any(word in kg for word in text.split())]
    return " ".join(hits[:topk])

# ✅ PyTorch Dataset
class PoetryDataset(Dataset):
    def __init__(self, dataframe, use_kg=False):
        self.data = dataframe.reset_index(drop=True)
        self.use_kg = use_kg

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['text']
        if self.use_kg:
            kg = find_relevant_kg(text, kg_texts)
            text = f"{text} [SEP] {kg}"
        inputs = tokenizer(
            text,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in inputs.items()}
        item["labels"] = torch.tensor(row["label"], dtype=torch.long)
        return item

# ✅ 데이터 분리 (8:2), stratify 가능
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# ✅ 모델 정의
def get_model():
    return BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    ).to(device)

# ✅ 학습 함수
def train_model(model, train_loader, epochs=3):
    optimizer = AdamW(model.parameters(), lr=5e-5)
    model.train()
    for epoch in range(epochs):
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(f"Epoch {epoch+1} 완료")

# ✅ 평가 함수
def evaluate(model, test_loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds += torch.argmax(logits, axis=1).cpu().tolist()
            labels += batch["labels"].cpu().tolist()
    print(classification_report(labels, preds, target_names=["human", "ai"]))


# ---- KG 미사용 실험 ----
model = get_model()
train_loader_no_kg = DataLoader(
    PoetryDataset(train_df, use_kg=False),
    batch_size=8,
    shuffle=True
)
test_loader_no_kg = DataLoader(
    PoetryDataset(test_df, use_kg=False),
    batch_size=8
)

print("🔥 KG 미사용 학습 시작")
train_model(model, train_loader_no_kg)
print("📊 KG 미사용 성능")
evaluate(model, test_loader_no_kg)


# ---- KG 사용 실험 ----
model = get_model()
train_loader_kg = DataLoader(
    PoetryDataset(train_df, use_kg=True),
    batch_size=8,
    shuffle=True
)
test_loader_kg = DataLoader(
    PoetryDataset(test_df, use_kg=True),
    batch_size=8
)

print("\n🔥 KG 사용 학습 시작")
train_model(model, train_loader_kg)
print("📊 KG 사용 성능")
evaluate(model, test_loader_kg)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [42]:
import pandas as pd
import torch
from torch.optim import AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import json

# 1) 데이터 로드
def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]
data = load_jsonl("poetry.jsonl")
df = pd.DataFrame(data)
print("❓ 로드 후 샘플 수:", len(df))
print("❓ 라벨 분포 (로드 직후):\n", df['label'].value_counts())

# 2) 문자열 'human'/'ai' → 0/1 매핑 (이미 숫자면 건너뛰기)
label_map = {"human": 0, "ai": 1}
if df['label'].dtype == object:
    df['label'] = df['label'].map(label_map)

# 3) 숫자형으로 변환 & 0·1 외 나머지(NaN 포함) 삭제
df['label'] = pd.to_numeric(df['label'], errors='coerce').astype('Int64')
df = df[df['label'].isin([0, 1])].reset_index(drop=True)
df['label'] = df['label'].astype(int)

print("✅ 정제 후 샘플 수:", len(df))
print("❓ 라벨 분포 (정제 후):\n", df['label'].value_counts())

# 4) KG 불러오기
kg_df = pd.read_csv("kg_triples_test.csv")
kg_texts = (
    kg_df
    .apply(lambda r: f"{r['subject']} {r['predicate']} {r['object']}", axis=1)
    .tolist()
)
def find_relevant_kg(text, kg_texts, topk=3):
    hits = [kg for kg in kg_texts if any(tok in kg for tok in text.split())]
    return " ".join(hits[:topk])

# 5) Dataset 정의
class PoetryDataset(Dataset):
    def __init__(self, df, tokenizer, use_kg=False):
        self.df = df
        self.tokenizer = tokenizer
        self.use_kg = use_kg

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['text']
        if self.use_kg:
            kg = find_relevant_kg(text, kg_texts)
            text = f"{text} [SEP] {kg}"
        inputs = self.tokenizer(
            text,
            padding='max_length',
            max_length=128,
            truncation=True,
            return_tensors='pt'
        )
        item = {k: v.squeeze(0) for k, v in inputs.items()}
        item['labels'] = torch.tensor(self.df.iloc[idx]['label'], dtype=torch.long)
        return item

# 6) 데이터 분리
train_df, test_df = train_test_split(
    df, test_size=0.2, stratify=df['label'], random_state=42
)

# 7) 토크나이저 및 모델 준비
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)
device = torch.device("cpu")
def get_model():
    return BertForSequenceClassification.from_pretrained(
        model_name, num_labels=2
    ).to(device)

# 8) 학습/평가 함수
def train_model(model, loader, epochs=3):
    optim = AdamW(model.parameters(), lr=5e-5)
    model.train()
    for e in range(epochs):
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss = model(**batch).loss
            loss.backward()
            optim.step(); optim.zero_grad()
        print(f"Epoch {e+1} 완료")

def evaluate(model, loader):
    model.eval()
    preds, labs = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            preds += torch.argmax(logits, axis=1).cpu().tolist()
            labs  += batch['labels'].cpu().tolist()
    print(classification_report(labs, preds, target_names=['human','ai']))

# 9) KG 미사용
model = get_model()
dl_train = DataLoader(PoetryDataset(train_df, tokenizer, use_kg=False),
                      batch_size=8, shuffle=True)
dl_test  = DataLoader(PoetryDataset(test_df,  tokenizer, use_kg=False),
                      batch_size=8)
print("🔥 KG 미사용 학습 시작")
train_model(model, dl_train)
print("📊 KG 미사용 결과")
evaluate(model, dl_test)

# 10) KG 사용
model = get_model()
dl_train = DataLoader(PoetryDataset(train_df, tokenizer, use_kg=True),
                      batch_size=8, shuffle=True)
dl_test  = DataLoader(PoetryDataset(test_df,  tokenizer, use_kg=True),
                      batch_size=8)
print("\n🔥 KG 사용 학습 시작")
train_model(model, dl_train)
print("📊 KG 사용 결과")
evaluate(model, dl_test)


❓ 로드 후 샘플 수: 945
❓ 라벨 분포 (로드 직후):
 label
1    756
0    189
Name: count, dtype: int64
✅ 정제 후 샘플 수: 945
❓ 라벨 분포 (정제 후):
 label
1    756
0    189
Name: count, dtype: int64


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔥 KG 미사용 학습 시작
Epoch 1 완료
Epoch 2 완료
Epoch 3 완료
📊 KG 미사용 결과


d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       human       0.00      0.00      0.00        38
          ai       0.80      1.00      0.89       151

    accuracy                           0.80       189
   macro avg       0.40      0.50      0.44       189
weighted avg       0.64      0.80      0.71       189



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔥 KG 사용 학습 시작
Epoch 1 완료
Epoch 2 완료
Epoch 3 완료
📊 KG 사용 결과
              precision    recall  f1-score   support

       human       0.00      0.00      0.00        38
          ai       0.80      1.00      0.89       151

    accuracy                           0.80       189
   macro avg       0.40      0.50      0.44       189
weighted avg       0.64      0.80      0.71       189



d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\User\lsm40\anaconda3\envs\min24\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
